<a href="https://colab.research.google.com/github/eaguilar98/python_portfolio/blob/main/deep_learning_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicción de Admisión de Estudiantes con Redes Neuronales usando Keras

Este ejercicio está inspirado en el Nanodegree de Inteligencia Artificial de Udacity. El objetivo es predecir si aceptamos a un estudiante a graduate studies (maestría, doctorado, etc) a UCLA basándonos en 3 datos:
* GRE Scores (Es un examen estandar en todo Estados Unidos).
* GPA Scores (Promedio de la carrera, suele ir de 1 a 4).
* Class Rank (El rango de la clase, es de 1 a 4).

El dataset lo puedes encontrar en: http://www.ats.ucla.edu/

Cargar Librerias Base

In [ ]:
import pandas               as pd
import numpy                as np
import matplotlib.pyplot    as plt
import seaborn              as sns


cargar información

In [ ]:
file='/content/students-data.csv'
df=pd.read_csv(file)

In [ ]:
df.head(5)

In [ ]:
X = np.array(df[["gre","gpa"]])

In [ ]:
def plot_points(data):
    
    # Vamos a cargar los dos features numéricos de este problema: GRE y GPA. Lo cargamos como un arreglo de NumPy.
    X = np.array(data[["gre","gpa"]])
    
    # También cargamos el valor que queremos predecir.
    y = np.array(data["admit"])
    
    # Separamos los features según fue aceptado o rechazado
    admitted = X[np.argwhere(y==1)]
    rejected = X[np.argwhere(y==0)]
    
    # Hacemos un scatter plot, donde el color rojo es rechazado y los celestes son aceptados.
    plt.scatter([s[0][0] for s in rejected], [s[0][1] for s in rejected], s = 25, color = 'red', edgecolor = 'k')
    plt.scatter([s[0][0] for s in admitted], [s[0][1] for s in admitted], s = 25, color = 'cyan', edgecolor = 'k')
    plt.xlabel('Test (GRE)')
    plt.ylabel('Grades (GPA)')

In [ ]:
plot_points(df)
plt.show()

Aunque no hay una relación tan clara, se puede decir que los estudiantes con mejores promedios y calificaciones en el examen son más probables de ser aceptados. De todos modos, la información no se puede separar tan fácil como en otros problemas que hemos enfrentado. Tal vez si usamos el tercer feature, el Rank, se separe mejor la información. Vamos a hacer 4 gráficas como la anterior, una para cada promedio.


In [ ]:
# Separamos el rank
data_rank1 = df[df["rank"]==1]
data_rank2 = df[df["rank"]==2]
data_rank3 = df[df["rank"]==3]
data_rank4 = df[df["rank"]==4]

# Mostramos los gráficos para cada uno
plot_points(data_rank1)
plt.title("Rank 1")
plt.show()

plot_points(data_rank2)
plt.title("Rank 2")
plt.show()

plot_points(data_rank3)
plt.title("Rank 3")
plt.show()

plot_points(data_rank4)
plt.title("Rank 4")
plt.show()

Paso 3: Modificamos la información para la red neuronal
One-hot encoding
Pandas ya tiene herramientas para hacer esto, por lo que vamos a aprovecharlas. No te preocupes mucho por el código, intenta enfocarte en lo que está ocurriendo.

La función get_dummies va a separar variables categóricas en muchas variables. Por ejemplo:

In [ ]:
pd.get_dummies(df['rank'], prefix='rank')[:10]

In [ ]:
# Make dummy variables for rank
one_hot_data = pd.concat([df, pd.get_dummies(df['rank'], prefix='rank')], axis=1)
one_hot_data[:5]

In [ ]:
# Drop the previous rank column
one_hot_data = one_hot_data.drop('rank', axis=1)

# Print the first 10 rows of our data
one_hot_data[:10]

### Escalando datos

Todo se ve bien, pero tenemos un problema. El GRE parece ir de 200 a 800, mientras que el GPA va de 1 a 4. El rango en el GRE es mucho más grande y esto puede causar problemas. En la siguiente celda vamos a escalar la información para que los features estén entre 0 y 1.

In [ ]:
# Copiamos la información
processed_data = one_hot_data[:]

In [ ]:
#print(processed_data.shape)
# o len(dataset)
# Escalamos ambas columnas
processed_data['gre'] = processed_data['gre']/800
processed_data['gpa'] = processed_data['gpa']/4.0
processed_data[:5]

### Separar los datos en Training y Testing Set

** Pregunta 2. ¿Por qué tenemos que separar en Training y Testing Set?**

En la siguiente celda separaremos en training y testing set. El testing set será el 10% de nuestra info total. No se preocupen mucho por el código, pero entiendan qué está pasando. Si corres la siguiente celda muchas veces, los resultados serán diferentes. Lo que estamos haciendo es agarrar el 90% de los datos utilizando ```np.random.choice```. Especificamos que no usamos remplazo (cuando sacamos un valor, no lo ponemos de vuelta). En la segunda línea lo que hacemos es agarrar los elementos y soltarlos (para el training y para el testing, respectivamente).

In [ ]:
sample = np.random.choice(processed_data.index, size=int(len(processed_data)*0.9), replace=False)
train_data, test_data = processed_data.iloc[sample], processed_data.drop(sample)

print("El número de training samples es", len(train_data))
print("El número de testing samples es", len(test_data))
print(train_data[:7])
print(test_data[:7])

### Separar data en features y labels

**Pregunta 3. ¿Cuál es la diferencia entre features y labels?**

Si te sale un error en la siguiente celda asegúrate de tener Keras instalado en tu ambiente.

In [ ]:
import keras
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

In [ ]:
# Soltamos la columna de admit en los features
features = np.array(train_data.drop('admit', axis=1))

In [ ]:
# Primero convertimos admit en dos columnas (one-hot encoding) usando la función to_categorical de Keras. 
# Esta es la que se usa normalmente para hacer one-hot encoding.
targets = np.array(tf.keras.utils.to_categorical(train_data['admit'], 2))

In [ ]:
# Hacemos lo mismo para el testing set
features_test = np.array(test_data.drop('admit', axis=1))
targets_test = np.array(tf.keras.utils.to_categorical(test_data['admit'], 2))

## Paso 4. Construimos Arquitectura del Modelo

In [ ]:
# Imports
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from tensorflow.keras.optimizers import SGD
from keras.utils import np_utils

In [ ]:
# Construimos el modelo
model = Sequential()
model.add(Dense(256, activation='sigmoid', input_shape=(6,)))

model.add(Dense(128, activation='sigmoid'))
model.add(Dense(128, activation='sigmoid'))
model.add(Dense(128, activation='sigmoid'))


model.add(Dense(2, activation='softmax'))
# Softmax en la ultima capa, nos arroja una probabilidad de pertenencia para c/u de las clases

In [ ]:
# Compilamos el modelo
model.compile(loss = 'categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
#BinaryCrossEntropy, Optimezer Adam
model.summary()

## Paso 5. Entrenamos el modelo

In [ ]:
model.fit(features, targets, epochs=100, batch_size=100, verbose=0)

## Paso 6. Evaluamos el modelo

In [ ]:
# Evaluating the model on the training and testing set

score = model.evaluate(features, targets)
print("\n Training Accuracy:", score[1])

score = model.evaluate(features_test, targets_test)
print("\n Testing Accuracy:", score[1])